### create tfrecord
1. resize image to RESIZE_TARGET_PATH
2. create tfrecord

need a little modify to use

### how to create kfold cav
https://www.kaggle.com/underwearfitting/how-to-properly-split-folds

In [ ]:
import tensorflow as tf
import pandas as pd
import cv2
from tqdm import tqdm
import os

In [ ]:
IMAGE_PATH = "../train/"
RESIZE_TARGET_PATH = "../train_600/"
TF_RECORD_PATH = "../train_600_tf_record/"
K_FOLDS = 5

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def resize_dataset():
    import glob
    values = glob.glob(f"{IMAGE_PATH}*.jpg")
    for i in tqdm(range(len(values))):
        img = cv2.imread(values[i])
        img = cv2.cv2.resize(img, (600, 600))
        cv2.imwrite(RESIZE_TARGET_PATH + values[i].split("/")[-1], img)

def serialize_example(columns, data):
    byte_features = ["StudyInstanceUID", 'PatientID']
    feature = {}
    for column in columns:
        if column in byte_features:
            feature[column] = _bytes_feature(data[columns.index(column)].encode('utf-8'))
        else:
            feature[column] = _int64_feature(data[columns.index(column)])

    feature["images"] = _bytes_feature(tf.io.gfile.GFile(f"{RESIZE_TARGET_PATH + data[columns.index('StudyInstanceUID')]}.jpg", 'rb').read())
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def create_tfrecord(df_fold, fold=0, phase="train"):
    data = df_fold.values
    columns = df_fold.columns.tolist()
    if not os.path.exists(TF_RECORD_PATH):
        os.makedirs(TF_RECORD_PATH)
    with tf.io.TFRecordWriter(f'{TF_RECORD_PATH}/{fold}_{phase}.tfrecord') as writer:
        for i in tqdm(range(len(data))):
            example = serialize_example(columns, data[i])
            writer.write(example)
    writer.close()


In [ ]:
if __name__ == '__main__':
    df_fold = pd.read_csv("train_folds.csv")
    for fold in range(K_FOLDS):
        print("===========================================")
        print(f"create fold tfrecord:   {fold}")
        print("create validation tfrecord")
        df_fold_val = df_fold[df_fold["fold"] == fold]
        create_tfrecord(df_fold_val,fold, phase="validate")
        print("create train tfrecord")
        df_fold_train = df_fold[df_fold["fold"] != fold]
        create_tfrecord(df_fold_train, fold)